In [1]:
import sys
sys.path.append("../src")
from mistral_client import run_mistral
from ner_post_processing import parse_entities_promptner, get_token_labels

import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from collections import defaultdict

/home/dachun/miniconda3/envs/mistral-ai-hackathon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("DFKI-SLT/cross_ner", "politics")

class_labels = dataset["validation"].features["ner_tags"].feature.names
index2label = {i: label for i, label in enumerate(class_labels)}
label2index = {v: k for k, v in index2label.items()}

label2index

{'O': 0,
 'B-academicjournal': 1,
 'I-academicjournal': 2,
 'B-album': 3,
 'I-album': 4,
 'B-algorithm': 5,
 'I-algorithm': 6,
 'B-astronomicalobject': 7,
 'I-astronomicalobject': 8,
 'B-award': 9,
 'I-award': 10,
 'B-band': 11,
 'I-band': 12,
 'B-book': 13,
 'I-book': 14,
 'B-chemicalcompound': 15,
 'I-chemicalcompound': 16,
 'B-chemicalelement': 17,
 'I-chemicalelement': 18,
 'B-conference': 19,
 'I-conference': 20,
 'B-country': 21,
 'I-country': 22,
 'B-discipline': 23,
 'I-discipline': 24,
 'B-election': 25,
 'I-election': 26,
 'B-enzyme': 27,
 'I-enzyme': 28,
 'B-event': 29,
 'I-event': 30,
 'B-field': 31,
 'I-field': 32,
 'B-literarygenre': 33,
 'I-literarygenre': 34,
 'B-location': 35,
 'I-location': 36,
 'B-magazine': 37,
 'I-magazine': 38,
 'B-metrics': 39,
 'I-metrics': 40,
 'B-misc': 41,
 'I-misc': 42,
 'B-musicalartist': 43,
 'I-musicalartist': 44,
 'B-musicalinstrument': 45,
 'I-musicalinstrument': 46,
 'B-musicgenre': 47,
 'I-musicgenre': 48,
 'B-organisation': 49,
 'I-o

In [5]:
fp = "../data/unlabeled/test_knn.csv"
fp_scored = "../data/scored/test_knn.csv"

In [20]:
df = pd.read_csv(fp, header=None, names=["src", "sim1", "sim2", "sim3", "sim4", "sim5"])

In [28]:
sentences = df["sim1"].to_list() + df["sim2"].to_list() + df["sim3"].to_list() + df["sim4"].to_list() + df["sim5"].to_list()
sentences = list(set(sentences))

In [29]:
prompt = lambda text: f"""
Dfn: An entity is a person (person), organisation (organisation), politician (politician), political party (politicalparty), event (event), election (election), 
country (country), location (location) or other political entity (misc). Dates, times, abstract concepts, adjectives, and verbs are not entities.

Example 1: Sitting as a Liberal Party of Canada Member of Parliament (MP) for Niagara Falls, she joined the Canadian Cabinet after the Liberals defeated the 
Progressive Conservative Party of Canada government of John Diefenbaker in the 1963 Canadian federal election.

Answer:
1. Liberal Party of Canada | True | as it is a political party (politicalparty)
2. Parliament | True | as it is an organisation (organisation)
3. Niagara Falls | True | as it is a location (location)
4. Canadian Cabinet | True | as it is a political entity (misc)
5. Liberals | True | as it is a political group by not the party name (misc)
6. Progressive Conservative Party of Canada | True | as it is a political party (politicalparty)
7. government | False | as it is not actually an entity in this sentence
8. John Diefenbaker | True | as it is a politician (politician)
9. 1963 Canadian federal election | True | as it is an election (election)

Example 2: The MRE took part to the consolidation of The Olive Tree as a joint electoral list both for the
2004 European Parliament election and the 2006 Italian general election, along with the Democrats of the Left
and Democracy is Freedom - The Daisy.

Answer:
1. MRE | True | as it is a political party (politicalparty)
2. consolidation | False | as it is an action
3. The Olive Tree | True | as it is a group or organisation (organisation)
4. 2004 European Parliament election | True | as it is an election (election)
5. 2006 Italian general election | True | as it is an election (election)
6. Democrats of the Left | True | as it is a political party (politicalparty)
7. Democracy is Freedom - The Daisy | True | as it is an political party (politicalparty)

Q. Given the paragraph below, identify a list of possible entities and for each entry explain why it either is or is not an entity.

Paragraph: {text}
"""

In [30]:
scored = defaultdict(list)

for idx, text in enumerate(tqdm(sentences)):
    if (idx + 1) % 100 == 0:
        df_scored = pd.DataFrame(scored)
        df_scored.to_csv(fp_scored, index=False)
    
    try:
        tokens = text.split()
        prompt_input = prompt(text)
        output = run_mistral(prompt_input)
        ner_tags = get_token_labels(text, parse_entities_promptner(output), label2index)

        scored["tokens"].append(text.split(" "))
        scored["prompt"].append(prompt_input)
        scored["output"].append(output)
        scored["ner_tags"].append(ner_tags)
    except Exception as e:
        print(e)
        continue

df_scored = pd.DataFrame(scored)
df_scored.to_csv(fp_scored, index=False)

 61%|██████    | 1700/2803 [2:26:43<12:17:00, 40.09s/it]

Unexpected exception (ReadTimeout): The read operation timed out


 78%|███████▊  | 2197/2803 [3:11:36<59:59,  5.94s/it]   

'float' object has no attribute 'split'


100%|██████████| 2803/2803 [4:06:51<00:00,  5.28s/it]  


In [35]:
df_scored

,tokens,prompt,output,ner_tags
0,"[Although, Ben-Gurion, formed, a, strong, coal...","\nDfn: An entity is a person (person), organis...",1. Ben-Gurion | True | as it is a politician (...,"[0, 57, 0, 0, 0, 0, 0, 0, 55, 56, 56, 0, 55, 5..."
1,"[He, also, served, on, nonprofit, boards, of, ...","\nDfn: An entity is a person (person), organis...",1. Guthrie Theater Foundation | True | as it i...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[On, 26, November, 2009, ,, in, an, interview,...","\nDfn: An entity is a person (person), organis...",1. 26 November 2009 | False | as it is a date\...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 51, 52, 0..."
3,"[Christabel, Pankhurst, was, the, daughter, of...","\nDfn: An entity is a person (person), organis...",1. Christabel Pankhurst | True | as it is a pe...,"[51, 52, 0, 0, 0, 0, 41, 42, 42, 42, 0, 51, 52..."
4,"[Founded, in, 1990, in, Boston, ,, Massachuset...","\nDfn: An entity is a person (person), organis...",1. 1990 | False | as it is a date\n2. Boston |...,"[0, 0, 0, 0, 35, 0, 35, 0, 0, 0, 0, 0, 0, 0, 3..."
...,...,...,...,...
2796,"[In, 1919, ,, at, the, Paris, Peace, Conferenc...","\nDfn: An entity is a person (person), organis...",1. 1919 | False | as it is a date\n2. Paris Pe...,"[0, 0, 0, 0, 0, 29, 30, 30, 0, 0, 0, 0, 0, 0, ..."
2797,"[She, ran, as, a, Conservative, in, Clarington...","\nDfn: An entity is a person (person), organis...",1. Conservative | True | as it is a political ...,"[51, 0, 0, 0, 55, 0, 35, 36, 36, 36, 36, 0, 0,..."
2798,"[One, year, later, ,, Le, Pen, announced, the,...","\nDfn: An entity is a person (person), organis...",1. One year later | False | as it is a time re...,"[0, 0, 0, 0, 57, 58, 0, 0, 0, 0, 55, 56, 56, 5..."
2799,"[Between, 1941, and, 1944, ,, the, German, inv...","\nDfn: An entity is a person (person), organis...",1. 1941 and 1944 | False | as they are dates\n...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 22, 0, 0, 0..."
